In [24]:
1+1

2

In [25]:
# 필요한 라이브러리 설치
!pip install transformers torch accelerate bitsandbytes
!pip install openai anthropic google-generativeai
!pip install pandas numpy matplotlib seaborn
!pip install datasets evaluate

In [26]:
# !apt-get install -y fonts-nanum
# !fc-cache -fv

# import json
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from datetime import datetime
# import os

# # 한글 폰트 설정
# plt.rcParams['font.family'] = 'NanumGothic'
# plt.rcParams['axes.unicode_minus'] = False

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings

warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = ['AppleGothic', 'Malgun Gothic', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 로드 완료")

✅ 라이브러리 로드 완료


In [27]:
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
# 2. 프롬프트 데이터 로드

def load_dongrae_prompts(file_path):
    """법무법인 동래 프롬프트 JSON 파일 로드"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            prompts = json.load(f)
        print(f"✅ 총 {len(prompts)}개 프롬프트 로드 완료")
        return prompts
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return None
    except Exception as e:
        print(f"❌ 파일 로드 오류: {str(e)}")
        return None

In [29]:
def load_dongrae_prompts(file_path):
    """법무법인 동래 프롬프트 JSON 파일 로드"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            prompts = json.load(f)
        print(f"✅ 총 {len(prompts)}개 프롬프트 로드 완료")
        return prompts
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return None
    except Exception as e:
        print(f"❌ 파일 로드 오류: {str(e)}")
        return None


In [30]:
json_file_path = "dongrae_simple_massive_20250701_180444.json"

In [31]:
prompts_data = load_dongrae_prompts(json_file_path)

if prompts_data is None:
    print("📝 JSON 파일이 없어서 샘플 프롬프트를 사용합니다.")
    prompts_data = [{"prompt": p, "query": p} for p in sample_prompts]

print(f"📊 사용할 프롬프트 수: {len(prompts_data)}개")

✅ 총 1000개 프롬프트 로드 완료
📊 사용할 프롬프트 수: 1000개


In [32]:
print("\n🔍 프롬프트 미리보기:")
for i, prompt in enumerate(prompts_data[:5], 1):
    if isinstance(prompt, dict):
        text = prompt.get('prompt', str(prompt))
    else:
        text = str(prompt)
    print(f"{i}. {text[:80]}{'...' if len(text) > 80 else ''}")


🔍 프롬프트 미리보기:
1. (최근5년) 기준 (부산) (소송및분쟁해결) 분야의 (파트너수)를 알려줘.
2. (2019-2023년) 기준 (부산) (IT통신법무) 변호사 (해결율) 평균은?
3. (2024년) 기준 (부산) (소송및분쟁해결) 변호사 (시간당비용) 평균은?
4. (지적재산권) 사건의 (서비스품질) 기준과 (부산) 지역 특성을 고려한 변호사 선택 가이드를 제공해주세요.
5. (부산)에서 (금융법무) 전문 변호사의 (해결율)와 (부산법원) 정보를 종합해서 설명해주세요.


In [33]:
def setup_gemini_api():
    """Google Gemini API 설정"""
    try:
        import google.generativeai as genai
        
        # API 키 입력
        api_key = input("🔑 Gemini API 키를 입력하세요 (없으면 엔터): ").strip()
        
        if not api_key:
            print("⏭️ Gemini API 키가 없어서 건너뜁니다.")
            return None
        
        genai.configure(api_key=OPENAI_API_KEY)
        model = genai.GenerativeModel('gemini-pro')
        
        # API 연결 테스트
        try:
            test_response = model.generate_content("안녕하세요")
            print("✅ Gemini API 연결 성공")
            return model
        except Exception as e:
            print(f"❌ Gemini API 연결 실패: {str(e)}")
            return None
            
    except ImportError:
        print("❌ google-generativeai 패키지가 설치되지 않았습니다.")
        print("💡 설치 명령어: pip install google-generativeai")
        return None


In [34]:
def test_with_openai(prompt, openai_client, model="gpt-4o-mini"):
    """OpenAI API로 프롬프트 테스트"""
    
    system_message = """당신은 법무법인 동래의 AI 법률 상담 어시스턴트입니다. 
법무법인 동래는 1995년 설립된 부산 연제구 소재 법무법인으로, 29년간 부산·경남 지역 법률 서비스를 제공해왔습니다.
부산 지역 특성을 반영한 전문적이고 친근한 법률 조언을 제공해주세요."""
    
    try:
        response = openai_client.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"오류 발생: {str(e)}"

In [35]:
test_prompt = prompts_data[0]['prompt']
print(f"🧪 테스트 프롬프트: {test_prompt}")
print("=" * 80)

🧪 테스트 프롬프트: (최근5년) 기준 (부산) (소송및분쟁해결) 분야의 (파트너수)를 알려줘.


In [36]:
test_results = {}

In [37]:
if 'openai_client' in locals() and openai_client is not None:
    print("🔮 OpenAI 모델 테스트 중...")
    openai_result = test_with_openai(test_prompt, openai_client)
    test_results["OpenAI"] = openai_result
    print(f"✅ OpenAI 결과: {openai_result[:100]}...")
    print()


In [38]:
def batch_test_prompts(prompts, test_function, model_name, max_tests=None):
    """프롬프트 배치 테스트"""
    if max_tests:
        prompts = prompts[:max_tests]
    
    results = []
    total = len(prompts)
    
    print(f"🚀 {model_name} 모델로 {total}개 프롬프트 테스트 시작...")
    
    for i, prompt_data in enumerate(prompts):
        if i % 5 == 0:
            print(f"📊 진행률: {i}/{total} ({i/total*100:.1f}%)")
        
        try:
            # 프롬프트 텍스트 추출
            if isinstance(prompt_data, dict):
                query = prompt_data.get('prompt', '')
                original_query = prompt_data.get('query', '')
            else:
                query = str(prompt_data)
                original_query = query
            
            # LLM 테스트
            start_time = datetime.now()
            response = test_function(query)
            end_time = datetime.now()
            
            response_time = (end_time - start_time).total_seconds()
            
            result = {
                "index": i,
                "original_query": original_query,
                "prompt": query,
                "response": response,
                "response_time": response_time,
                "response_length": len(response),
                "timestamp": datetime.now().isoformat(),
                "model": model_name,
                "success": True
            }
            
        except Exception as e:
            result = {
                "index": i,
                "original_query": original_query if 'original_query' in locals() else "",
                "prompt": query if 'query' in locals() else str(prompt_data),
                "response": "",
                "response_time": 0,
                "response_length": 0,
                "error": str(e),
                "timestamp": datetime.now().isoformat(),
                "model": model_name,
                "success": False
            }
        
        results.append(result)
    
    print(f"✅ {model_name} 테스트 완료!")
    return results

In [39]:
# 배치 테스트 실행 여부 선택
run_batch_test = input("📝 배치 테스트를 실행하시겠습니까? (y/n): ").lower() == 'y'

if run_batch_test:
    max_tests = int(input("🔢 테스트할 프롬프트 수를 입력하세요 (전체는 0): ") or "10")
    if max_tests == 0:
        max_tests = None
    
    all_batch_results = []
    
    # 각 모델별 배치 테스트
    # if 'tokenizer' in locals() and tokenizer is not None:
    #     hf_test_func = lambda p: test_with_huggingface(p, tokenizer, hf_model, device)
    #     hf_results = batch_test_prompts(prompts_data, hf_test_func, "Hugging Face", max_tests)
    #     all_batch_results.extend(hf_results)
    
    if 'openai_client' in locals() and openai_client is not None:
        openai_test_func = lambda p: test_with_openai(p, openai_client)
        openai_results = batch_test_prompts(prompts_data, openai_test_func, "OpenAI", max_tests)
        all_batch_results.extend(openai_results)
    
    # if 'gemini_model' in locals() and gemini_model is not None:
    #     gemini_test_func = lambda p: test_with_gemini(p, gemini_model)
    #     gemini_results = batch_test_prompts(prompts_data, gemini_test_func, "Gemini", max_tests)
    #     all_batch_results.extend(gemini_results)
    
    print(f"🎉 전체 배치 테스트 완료! 총 {len(all_batch_results)}개 결과")

In [40]:
def analyze_test_results(results):
    """테스트 결과 분석 및 시각화"""
    if not results:
        print("❌ 분석할 결과가 없습니다.")
        return None
    
    df = pd.DataFrame(results)
    
    print("📊 === 테스트 결과 분석 ===")
    print(f"총 테스트 수: {len(results)}")
    
    # 모델별 분석
    if 'model' in df.columns:
        model_stats = df.groupby('model').agg({
            'success': ['count', 'sum', 'mean'],
            'response_length': 'mean',
            'response_time': 'mean'
        }).round(2)
        print("\n🤖 모델별 성능:")
        print(model_stats)
    
    # 전체 통계
    success_rate = df['success'].mean() * 100
    avg_length = df['response_length'].mean()
    avg_time = df['response_time'].mean()
    
    print(f"\n📈 전체 통계:")
    print(f"성공률: {success_rate:.1f}%")
    print(f"평균 응답 길이: {avg_length:.1f}자")
    print(f"평균 응답 시간: {avg_time:.2f}초")
    
    # 시각화
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. 성공률
    if 'model' in df.columns and len(df['model'].unique()) > 1:
        model_success = df.groupby('model')['success'].mean()
        axes[0,0].bar(model_success.index, model_success.values)
        axes[0,0].set_title('모델별 성공률')
        axes[0,0].set_ylabel('성공률')
    else:
        axes[0,0].pie([df['success'].sum(), len(df) - df['success'].sum()], 
                     labels=['성공', '실패'], autopct='%1.1f%%')
        axes[0,0].set_title('전체 성공률')
    
    # 2. 응답 길이 분포
    axes[0,1].hist(df['response_length'].dropna(), bins=20, alpha=0.7, color='skyblue')
    axes[0,1].set_xlabel('응답 길이 (글자수)')
    axes[0,1].set_ylabel('빈도')
    axes[0,1].set_title('응답 길이 분포')
    
    # 3. 프롬프트 길이 vs 응답 길이
    df['prompt_length'] = df['prompt'].str.len()
    axes[1,0].scatter(df['prompt_length'], df['response_length'], alpha=0.6, color='green')
    axes[1,0].set_xlabel('프롬프트 길이')
    axes[1,0].set_ylabel('응답 길이')
    axes[1,0].set_title('프롬프트 길이 vs 응답 길이')
    
    # 4. 응답 시간 분포
    axes[1,1].hist(df['response_time'].dropna(), bins=20, alpha=0.7, color='orange')
    axes[1,1].set_xlabel('응답 시간 (초)')
    axes[1,1].set_ylabel('빈도')
    axes[1,1].set_title('응답 시간 분포')
    
    plt.tight_layout()
    plt.show()
    
    return df

In [41]:
if 'all_batch_results' in locals() and all_batch_results:
    analysis_df = analyze_test_results(all_batch_results)
elif 'test_results' in locals() and test_results:
    # 단일 테스트 결과를 DataFrame으로 변환
    single_results = []
    for model, response in test_results.items():
        single_results.append({
            "model": model,
            "prompt": test_prompt,
            "response": response,
            "response_length": len(response),
            "success": "오류" not in response
        })
    analysis_df = analyze_test_results(single_results)
else:
    print("⚠️ 분석할 테스트 결과가 없습니다.")


⚠️ 분석할 테스트 결과가 없습니다.


In [42]:
def save_results(results, filename_prefix="dongrae_llm_test"):
    """테스트 결과 저장"""
    if not results:
        print("❌ 저장할 결과가 없습니다.")
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # JSON 파일로 저장
    json_filename = f"{filename_prefix}_{timestamp}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    # CSV 파일로 저장 (분석용)
    df = pd.DataFrame(results)
    csv_filename = f"{filename_prefix}_{timestamp}.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"💾 결과 저장 완료:")
    print(f"   📄 JSON: {json_filename}")
    print(f"   📊 CSV: {csv_filename}")
    
    return json_filename, csv_filename

In [43]:
if 'all_batch_results' in locals() and all_batch_results:
    save_results(all_batch_results, "dongrae_batch_test")


In [44]:
# def generate_quality_report(df):
#     """품질 리포트 생성"""
#     if df is None or df.empty:
#         print("❌ 리포트 생성할 데이터가 없습니다.")
#         return
    
#     report = f"""
# # 🏢 법무법인 동래 프롬프트 테스트 품질 리포트

In [45]:
# # 🏢 법무법인 동래 프롬프트 테스트 품질 리포트

# ## 📊 전체 통계
# - **총 테스트 수**: {len(df)}개
# - **성공률**: {df['success'].mean()*100:.1f}%
# - **평균 응답 길이**: {df['response_length'].mean():.1f}자
# - **평균 응답 시간**: {df['response_time'].mean():.2f}초

# ## 🎯 품질 지표
# - **적절한 길이 응답 (100-500자)**: {((df['response_length'] >= 100) & (df['response_length'] <= 500)).mean()*100:.1f}%
# - **짧은 응답 (<100자)**: {(df['response_length'] < 100).mean()*100:.1f}%
# - **긴 응답 (>500자)**: {(df['response_length'] > 500).mean()*100:.1f}%

# ## 🚀 모델별 성능
# """
    
#     if 'model' in df.columns:
#         for model in df['model'].unique():
#             model_df = df[df['model'] == model]
#             report += f"""
# ### {model}
# - 성공률: {model_df['success'].mean()*100:.1f}%
# - 평균 응답 길이: {model_df['response_length'].mean():.1f}자
# - 평균 응답 시간: {model_df['response_time'].mean():.2f}초
# """
    
#     report += f"""
# ## 💡 추천사항
# {"1. 프롬프트 길이를 조정하여 더 구체적인 응답 유도 필요" if df['response_length'].mean() < 200 else "1. 현재 응답 길이가 적절함"}
# {"2. 오류율이 높으니 프롬프트 구조 개선 필요" if df['success'].mean() < 0.8 else "2. 안정적인 성능을 보이고 있음"}
# 3. 법무법인 동래 특화 컨텍스트 강화 고려
# 4. 부산 지역 특성을 더 잘 반영하는 프롬프트 개발
# 5. 응답 품질 개선을 위한 추가 fine-tuning 검토

# ## 📋 결론
# {"전반적으로 만족스러운 성능을 보이고 있으며, 법무법인 동래의 AI 서비스에 활용 가능한 수준입니다." if df['success'].mean() > 0.8 else "성능 개선이 필요하며, 프롬프트 엔지니어링과 모델 튜닝을 통한 최적화가 권장됩니다."}
# """
    
#     print(report)
    
#     # 리포트 파일로 저장
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     report_filename = f"dongrae_quality_report_{timestamp}.md"
#     with open(report_filename, 'w', encoding='utf-8') as f:
#         f.write(report)
    
#     print(f"📋 품질 리포트 저장: {report_filename}")
#     return report

# # 품질 리포트 생성
# if 'analysis_df' in locals() and analysis_df is not None:
#     quality_report = generate_quality_report(analysis_df)
